# Import



In [1]:
import time
#from polygon import WebSocketClient, STOCKS_CLUSTER
import requests
import pandas as pd
import json
import datetime
import pytz
import dateutil
import logging

#Show all columns when printing dataframes
pd.set_option('display.max_columns', None)

# Get data

In [2]:
key = '3U413sHUAdFisFvcp6TReoTsEZ_GgpLp'
response = requests.get('https://api.polygon.io/v2/aggs/ticker/AAPL/range/15/minute/2021-01-10/2021-01-15?unadjusted=true&sort=asc&limit=10000&apiKey=3U413sHUAdFisFvcp6TReoTsEZ_GgpLp')
data = json.loads(response.text)

# Create DataFrame and add datetime column

In [3]:
df = pd.DataFrame(data['results'])

for idx,row in df.iterrows():
    df.at[idx, 'date_time'] = datetime.datetime.utcfromtimestamp(row['t']/1000) - datetime.timedelta(hours=5) #.astimezone(dateutil.tz.gettz('US/Eastern'))#.strftime('%Y-%m-%d %H:%M:%S')

print(df[df['date_time'] == '2021-01-15 15:59:00'])
#print(df.head())

Empty DataFrame
Columns: [v, vw, o, c, h, l, t, n, date_time]
Index: []


# Remove ETH data

In [4]:
opening_hours = datetime.time(9, 30)
closing_hours = datetime.time(16, 00)

print(f'Amout of data: {len(df)}')

idx_rem = []
for idx, row in df.iterrows():
    if row['date_time'].time() >= opening_hours and row['date_time'].time() <= closing_hours: continue
    idx_rem.append(idx)
df.drop(idx_rem, inplace=True)
#print(df[ df['datetime'] >= opening_hours & df['datetime'] <= closing_hours ] )
#print(df)
print(f'Amout of RTH only data: {len(df)}')
print(df)
# start_date = datetime.datetime(2021, 1, 4, 0, 0, 0)
# end_date = datetime.datetime(2021, 1, 4, 23, 59, 59)
# mask = (df['date_time'] >= start_date) & (df['date_time'] < end_date)


# print(df.loc[mask])

Amout of data: 320
Amout of RTH only data: 135
              v        vw         o         c         h        l  \
22   11657184.0  129.0485  129.1900  129.3937  129.8200  128.500   
23    5981867.0  129.0660  129.3939  129.4250  129.5300  128.600   
24    5809319.0  129.7965  129.4100  129.6900  130.1700  129.370   
25    3748006.0  129.8083  129.6900  130.0000  130.0800  129.580   
26    3420726.0  129.8739  130.0000  129.6350  130.1100  129.550   
..          ...       ...       ...       ...       ...      ...   
300   3225303.0  127.6442  127.7400  127.8100  127.8457  127.490   
301   2458301.0  127.7631  127.8100  127.5600  127.9700  127.545   
302   3903295.0  127.4355  127.5600  127.1900  127.6100  127.180   
303   7074465.0  127.2327  127.1800  127.0700  127.5000  127.000   
304   3907884.0  127.1828  127.1400  127.1400  128.4798  126.910   

                 t      n           date_time  
22   1610375400000  90078 2021-01-11 09:30:00  
23   1610376300000  60123 2021-01-11 09:

# Add SMA


In [5]:
sma_period = 30
c_idx = df.columns.get_loc("c")
df['sma300'] = df.iloc[:,c_idx].rolling(window=sma_period).mean()
print(df[df['date_time'] == '2021-01-15 13:00:00'])

             v        vw       o       c       h       l              t  \
292  1576119.0  128.3695  128.48  128.52  128.52  128.27  1610733600000   

         n           date_time      sma300  
292  12943 2021-01-15 13:00:00  129.059793  
